# Read POIs and Map them to SA2 areas

1. process POIs by mapping them to a small number of categories.
2. normalize counts and entropy by the size of SA2s.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
# import geoplot
import pickle
# import geoplot.crs as gcrs
import statsmodels.api as sm

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
node_degree_df = pd.read_pickle('../../data/processed_small_data/node_degree_df.pickle')
edge_degree_df = pd.read_pickle('../../data/processed_small_data/edge_degree_df.pickle')

In [3]:
len(node_degree_df["SA2_MAIN16"].values)

103

In [4]:
edge_degree_df.columns

Index(['timestamp', 'destination', 'state', 'origin', 'unique_agents',
       'sum_stay_duration', 'total_stays', 'OD', 'class_ART', 'class_BUS',
       'class_COLL', 'class_FREE', 'class_HWY', 'class_LOCL', 'class_SUBA',
       'class_TRK2', 'class_TRK4', 'class_UND', 'num_jobs_000_persons_origin',
       'sa2_code16_x', 'median_income_per_job_aud_origin',
       'num_jobs_000_persons_destination', 'sa2_code16_y',
       'median_income_per_job_aud_destination', 'od_duration_value',
       'od_distance_value', 'road_counts', 'num_nodes_x', 'num_1degree_x',
       'num_2degree_x', 'num_3degree_x', 'num_4degree_x',
       'num_greater5degree_x', 'num_nodes_y', 'num_1degree_y', 'num_2degree_y',
       'num_3degree_y', 'num_4degree_y', 'num_greater5degree_y'],
      dtype='object')

In [5]:
# mount_path = "/mnt/c/Users/jason/Dropbox (MIT)/"
mount_path = "/Users/shenhaowang/Dropbox (MIT)/project_econ_opportunity_south_Australia/"

In [6]:
poi_df = gpd.read_file(mount_path + "/SA data/dataSA/points/points.shp")

In [7]:
poi_df.head()

,osm_id,timestamp,name,type,geometry
0,29240333,None,None,traffic_signals,POINT (138.60119 -34.91082)
1,29398542,None,None,crossing,POINT (138.59423 -34.76718)
2,29491931,None,Adelaide Railway Station Tram Stop (To Adelaide,tram_stop,POINT (138.59739 -34.92171)
3,29491942,None,None,level_crossing,POINT (138.59949 -34.92299)
4,29492063,None,None,traffic_signals,POINT (138.59351 -34.92330)


In [8]:
sa2_south_au = gpd.read_file("../../data/processed_small_data/shapefiles/sa2_south_au.shp")

In [9]:
sa2_south_au.shape

(172, 13)

In [10]:
sa2_south_au.head()

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
0,401011001,41001,Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,10.4824,"POLYGON ((138.58096 -34.93024, 138.58094 -34.9..."
1,401011002,41002,North Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,5.0909,"POLYGON ((138.58214 -34.90424, 138.58287 -34.9..."
2,401021003,41003,Adelaide Hills,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,364.4390,"POLYGON ((138.75610 -34.82107, 138.75567 -34.8..."
3,401021004,41004,Aldgate - Stirling,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,117.2141,"POLYGON ((138.69440 -34.98807, 138.69514 -34.9..."
4,401021005,41005,Hahndorf - Echunga,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,South Australia,110.1516,"POLYGON ((138.78877 -35.02474, 138.78889 -35.0..."


In [11]:
print(len(poi_df.geometry))

27236


In [12]:
poi_df

,osm_id,timestamp,name,type,geometry
0,29240333,None,None,traffic_signals,POINT (138.60119 -34.91082)
1,29398542,None,None,crossing,POINT (138.59423 -34.76718)
2,29491931,None,Adelaide Railway Station Tram Stop (To Adelaide,tram_stop,POINT (138.59739 -34.92171)
3,29491942,None,None,level_crossing,POINT (138.59949 -34.92299)
4,29492063,None,None,traffic_signals,POINT (138.59351 -34.92330)
...,...,...,...,...,...
27231,1665329,None,Cibo,cafe,POINT (138.74937 -34.59956)
27232,1665330,None,Pizza Baa,restaurant,POINT (138.74951 -34.59955)
27233,1665425,None,Westpac,bank,POINT (138.74936 -34.59943)
27234,2165175,None,None,information,POINT (138.94915 -34.48722)


In [19]:
# assign POIs to the SA2's
# it takes about 10~15 min.
sa_codes_poi = []
for i, centroid in enumerate(poi_df.geometry):
    if i%100 == 0: print(i)
    found = False
    for i, row in sa2_south_au.iterrows():
        if row["geometry"].contains(centroid):
            sa_codes_poi.append(row["SA2_MAIN16"])
            found = True
            break
    if not found:
        sa_codes_poi.append("0")
        
poi_df["SA2_MAIN16"] = sa_codes_poi
poi_df.to_pickle("../../data/processed_small_data/poi_df.pickle")


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [13]:
# read the file again
poi_df = pd.read_pickle("../../data/processed_small_data/poi_df.pickle")

In [8]:
# np.sum(poi_df["SA2_MAIN16"] == '0')

In [14]:
print(len(poi_df))
poi_df = poi_df[poi_df["SA2_MAIN16"]!="0"]
print(len(poi_df))

27236
27210


In [15]:
poi_df.head()

,osm_id,timestamp,name,type,geometry,SA2_MAIN16
0,29240333,None,None,traffic_signals,POINT (138.60119 -34.91082),401011002
1,29398542,None,None,crossing,POINT (138.59423 -34.76718),402041044
2,29491931,None,Adelaide Railway Station Tram Stop (To Adelaide,tram_stop,POINT (138.59739 -34.92171),401011001
3,29491942,None,None,level_crossing,POINT (138.59949 -34.92299),401011001
4,29492063,None,None,traffic_signals,POINT (138.59351 -34.92330),401011001


## Analyze three SA2s: 401011001 (Downtown), 401021004 (Suburban town), 401021008 (mountain?)

Note: the small towns and rural areas tend to have long-tail distribution of activities. Need to remove them.

In [20]:
# compute entropy.
def compute_entropy(list_):
    val = list_/np.sum(list_)
    entropy = np.sum(-val * np.log(val))
    return entropy

# print(compute_entropy([1,1,1]))
# print(compute_entropy([1,1,1,1,10]))


In [19]:
pd.set_option("display.max_rows", None)

# 1. urban center
print(np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401011001', 'type'], return_counts = True))
_, list_ = np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401011001', 'type'], return_counts = True)
# print(compute_entropy(list_))
print(pd.DataFrame([_,list_]).T)

# round 1: 2.48
# round 2: 2.43
# round 3: 1.55

# 2. periphery
print(np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401021008', 'type'], return_counts = True))
_, list_ = np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401021008', 'type'], return_counts = True)
# print(compute_entropy(list_))
print(pd.DataFrame([_,list_]).T)

# round 1: 2.99
# round 2: 2.95
# round 3: 1.50

# 3. suburban town
print(np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401021004', 'type'], return_counts = True))
_, list_ = np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401021004', 'type'], return_counts = True)
# print(compute_entropy(list_))
print(pd.DataFrame([_,list_]).T)

# round 1: 2.73
# round 2: 2.69
# round 3: 1.96


(array(['apartment', 'arts_centre', 'artwork', 'atm', 'attraction', 'bank',
       'bar', 'bbq', 'bench', 'bicycle_parking', 'bicycle_rental',
       'bicycle_repair_s', 'buffer_stop', 'bureau_de_change',
       'bus_station', 'bus_stop', 'cafe', 'car_rental',
       'charging_station', 'childcare', 'cinema', 'clinic',
       'community_centre', 'construction', 'crossing', 'dentist',
       'device_charging_', 'doctors', 'dojo', 'drinking_water',
       'events_venue', 'fast_food', 'food_court', 'fountain', 'fuel',
       'give_way', 'guest_house;hote', 'hostel', 'hotel', 'ice_cream',
       'information', 'level_crossing', 'library', 'memorial',
       'monitoring_stati', 'monument', 'motel', 'museum', 'nightclub',
       'parking', 'parking_entrance', 'parking_space', 'pharmacy',
       'place_of_worship', 'police', 'post_box', 'post_office', 'pub',
       'restaurant', 'restaurant;bar', 'school', 'shelter',
       'social_facility', 'speed_camera', 'station', 'steps', 'stripclub',
 

In [24]:
# reorganize the POIs.
names,hist=np.unique(poi_df['type'], return_counts = True)
# pd.set_option("display.max_rows", None, "display.max_columns", None)
hist_poi = pd.DataFrame([names,hist], index = ['names','counts']).T
hist_poi.sort_values('counts', ascending = False)


,names,counts
23,bus_stop,7356
157,turning_circle,4556
44,crossing,3947
14,bench,1999
151,traffic_signals,1666
15,bicycle_parking,609
131,storage_tank,567
79,level_crossing,558
115,restaurant,548
53,fast_food,405


In [25]:
# mapping POIs to a small number of categories.
drop_list = ['Historical_House',
             'adit','alpine_hut','animal_boarding','antenna','apartment','beacon','bench','bicycle_parking',
             'buffer_stop','building','bus_station','bus_stop',
             'chalet', 'charging_station', 'chimney', 'clock',
             'communications_t','compressed_air','construction','crossing',
             'device_charging_','disused','dojo', 'drinking_water',
             'elevator',
             'flagpole','fuel','funeral_home',
             'garbage_can','give_way','goods_conveyor','guest_house;hote','grave_yard','guest_house','guest_house;hote',
             'halt',
             'kiln',
             'lamp','level_crossing','loading_dock',
             'manhole', 'mast', 'milestone', 'mine', 'mine_shaft', 'motorway_junctio', 
             'parking', 'parking_entrance', 'parking_space', 'proposed', 
             'rest_area', 
             'sanitary_dump_st', 'silo', 'speed_camera','station','steps','stop','street_cabinet',
             'street_lamp','subway_entrance','surveillance','survey_point','swings','switch',
             'tank','taxi','tomb','tower', 
             'traffic_signals','traffic_signals;','trailhead','tram_level_cross','tram_stop','tree','turning_circle','turning_loop','turntable',
             'waste_basket','waste_transfer_s','wastewater_plant','water_point','water_tank','water_tap',
             'water_tower','water_well','windmill','windpump','wreck',
             'yes']

education = ['childcare', 'college', 'community_centre', 'kindergarten', 'music_school', 'school', 'university']

tour = ['attraction', 'castle', 'ferry_terminal', 'monument', 
        'picnic_site', 'place_of_worship', 'viewpoint', 'zoo']

restaurant = ['bar', 'bbq', 'cafe', 'fast_food', 'food_court', 'ice_cream', 'pub', 'restaurant', 'restaurant;bar']

culture = ['arts_centre', 'artwork', 'gallery', 'library', 'memorial', 'museum', 
           'piano', 'public_bookcase', 'ruins', 'shower', 'studio', 'theatre']

recreation = ['camp_pitch', 'camp_site', 'caravan_site', 'cinema', 'events_venue', 
              'fountain', 'nightclub', 'stripclub', 'swimming_pool']

small_business = ['bicycle_rental', 'bicycle_repair_s', 'brothel', 'car_rental', 'car_wash', 'gambling', 'makerspace','marketplace',
                  'vending_machine','veterinary','winery']

hotel = ['hostel', 'hotel', 'motel']

information = ['information', 'monitoring_stati', 'newsagency', 'telephone']

government = ['bureau_de_change', 'courthouse', 'fire_station', 'police', 'post_box', 'post_office', 'prison', 'pumping_station', 
              'recycling', 'scout_hall', 'scrapyard', 'shelter', 'shelter;drinking', 'social_facility', 'storage_tank',
              'toilets','townhall']

medical = ['clinic', 'dentist', 'doctors', 'first_aid', 'hospital', 'pharmacy', 'surgery']

finance = ['atm', 'bank']

# replacement dictionary
replacement_dict = dict(zip(drop_list+education+tour+restaurant+culture+recreation+small_business+hotel+information+government+medical+finance, 
                            ['drop']*len(drop_list)+['education']*len(education)+['tour']*len(restaurant)+['restaurant']*len(restaurant)+['culture']*len(culture)+\
                            ['recreation']*len(recreation)+['small_business']*len(small_business)+['hotel']*len(hotel)+['information']*len(information)+\
                            ['government']*len(government)+['medical']*len(medical)+['finance']*len(finance)))

replacement_dict


{'Historical_House': 'drop',
 'adit': 'drop',
 'alpine_hut': 'drop',
 'animal_boarding': 'drop',
 'antenna': 'drop',
 'apartment': 'drop',
 'beacon': 'drop',
 'bench': 'drop',
 'bicycle_parking': 'drop',
 'buffer_stop': 'drop',
 'building': 'drop',
 'bus_station': 'drop',
 'bus_stop': 'drop',
 'chalet': 'drop',
 'charging_station': 'drop',
 'chimney': 'drop',
 'clock': 'drop',
 'communications_t': 'drop',
 'compressed_air': 'drop',
 'construction': 'drop',
 'crossing': 'drop',
 'device_charging_': 'drop',
 'disused': 'drop',
 'dojo': 'drop',
 'drinking_water': 'drop',
 'elevator': 'drop',
 'flagpole': 'drop',
 'fuel': 'drop',
 'funeral_home': 'drop',
 'garbage_can': 'drop',
 'give_way': 'drop',
 'goods_conveyor': 'drop',
 'guest_house;hote': 'drop',
 'grave_yard': 'drop',
 'guest_house': 'drop',
 'halt': 'drop',
 'kiln': 'drop',
 'lamp': 'drop',
 'level_crossing': 'drop',
 'loading_dock': 'drop',
 'manhole': 'drop',
 'mast': 'drop',
 'milestone': 'drop',
 'mine': 'drop',
 'mine_shaft':

In [26]:
# replace
poi_df['type'].replace(to_replace = replacement_dict, inplace = True)

In [27]:
np.unique(poi_df['type'], return_counts = True)

(array(['culture', 'drop', 'education', 'finance', 'government', 'hotel',
        'information', 'medical', 'recreation', 'restaurant',
        'small_business', 'tour'], dtype=object),
 array([  232, 22538,    88,   104,  1414,   127,   122,   304,   131,
         1667,   122,   361]))

In [31]:
# save the shapefiles to 
poi_df.to_pickle("../../data/processed_small_data/poi_small_categories.pickle")


In [19]:
# drop the useless POIs
poi_df = poi_df.loc[np.logical_not(poi_df['type']=='drop'),:]
print(poi_df.shape)

(4672, 6)


In [21]:
# 1. urban center
print(np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401011001', 'type'], return_counts = True))
_, list_ = np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401011001', 'type'], return_counts = True)
print(compute_entropy(list_))
print(pd.DataFrame([_,list_]).T)

# round 1: 2.48
# round 2: 2.43
# round 3: 1.55


(array(['culture', 'education', 'finance', 'government', 'hotel',
       'information', 'medical', 'recreation', 'restaurant',
       'small_business', 'tour'], dtype=object), array([ 28,   5,  24,  56,  20,  13,  23,  25, 382,  20,  47]))
1.5502638418085137
                 0    1
0          culture   28
1        education    5
2          finance   24
3       government   56
4            hotel   20
5      information   13
6          medical   23
7       recreation   25
8       restaurant  382
9   small_business   20
10            tour   47


In [22]:
# 2. periphery
print(np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401021008', 'type'], return_counts = True))
_, list_ = np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401021008', 'type'], return_counts = True)
print(compute_entropy(list_))
print(pd.DataFrame([_,list_]).T)

# round 1: 2.99
# round 2: 2.95
# round 3: 1.50

(array(['culture', 'government', 'hotel', 'information', 'recreation',
       'restaurant', 'tour'], dtype=object), array([ 3, 19,  2,  1,  3,  6,  3]))
1.5036556861938575
             0   1
0      culture   3
1   government  19
2        hotel   2
3  information   1
4   recreation   3
5   restaurant   6
6         tour   3


In [23]:
# 3. suburban town
print(np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401021004', 'type'], return_counts = True))
_, list_ = np.unique(poi_df.loc[poi_df.SA2_MAIN16 == '401021004', 'type'], return_counts = True)
print(compute_entropy(list_))
print(pd.DataFrame([_,list_]).T)

# round 1: 2.73
# round 2: 2.69
# round 3: 1.96


(array(['culture', 'education', 'finance', 'government', 'hotel',
       'information', 'medical', 'restaurant', 'small_business', 'tour'],
      dtype=object), array([10, 10,  4, 30,  6,  3, 12, 36,  2, 11]))
1.9632823851262005
                0   1
0         culture  10
1       education  10
2         finance   4
3      government  30
4           hotel   6
5     information   3
6         medical  12
7      restaurant  36
8  small_business   2
9            tour  11


In [24]:
poi_df

,osm_id,timestamp,name,type,geometry,SA2_MAIN16
107,33109203,None,Immanuel Primary School,education,POINT (138.53700 -34.97254),404031107
135,57881377,None,The Feathers Hotel,restaurant,POINT (138.66033 -34.93764),401031011
136,57884413,None,Burnside Post Office,government,POINT (138.66022 -34.93661),401031011
137,57893666,None,None,government,POINT (138.67134 -34.92471),401031011
138,57919090,None,Burnside Library,culture,POINT (138.64364 -34.93866),401031013
...,...,...,...,...,...,...
27231,1665329,None,Cibo,restaurant,POINT (138.74937 -34.59956),402011026
27232,1665330,None,Pizza Baa,restaurant,POINT (138.74951 -34.59955),402011026
27233,1665425,None,Westpac,finance,POINT (138.74936 -34.59943),402011026
27234,2165175,None,None,hotel,POINT (138.94915 -34.48722),405011111


## Compute counts and entropy

In [27]:
count = poi_df.groupby(["SA2_MAIN16"],as_index=False).aggregate(["count"])

In [28]:
count.head()

,osm_id,timestamp,name,type,geometry
,count,count,count,count,count
SA2_MAIN16,,,,,
401011001,643,0,499,643,643
401011002,113,0,71,113,113
401021003,42,0,12,42,42
401021004,124,0,85,124,124
401021005,50,0,40,50,50


In [29]:
# sw Q: How/why do we miss so many SA2's here?
count.shape

(124, 5)

In [30]:
count.columns

MultiIndex([(   'osm_id', 'count'),
            ('timestamp', 'count'),
            (     'name', 'count'),
            (     'type', 'count'),
            ( 'geometry', 'count')],
           )

In [31]:
split_count = poi_df.groupby(["SA2_MAIN16","type"],as_index=False).aggregate(["count"])

In [32]:
split_count

osm_id timestamp  name geometry
                       count     count count    count
SA2_MAIN16 type                                      
401011001  culture        28         0    23       28
           education       5         0     5        5
           finance        24         0    24       24
           government     56         0    15       56
           hotel          20         0    17       20
...                      ...       ...   ...      ...
407031169  culture         1         0     1        1
           government      2         0     0        2
           recreation      4         0     3        4
           restaurant      7         0     4        7
           tour            5         0     1        5

[732 rows x 4 columns]

In [33]:
for i, row in split_count.iterrows():
    print(row)

osm_id     count    28
timestamp  count     0
name       count    23
geometry   count    28
Name: (401011001, culture), dtype: int64
osm_id     count    5
timestamp  count    0
name       count    5
geometry   count    5
Name: (401011001, education), dtype: int64
osm_id     count    24
timestamp  count     0
name       count    24
geometry   count    24
Name: (401011001, finance), dtype: int64
osm_id     count    56
timestamp  count     0
name       count    15
geometry   count    56
Name: (401011001, government), dtype: int64
osm_id     count    20
timestamp  count     0
name       count    17
geometry   count    20
Name: (401011001, hotel), dtype: int64
osm_id     count    13
timestamp  count     0
name       count     2
geometry   count    13
Name: (401011001, information), dtype: int64
osm_id     count    23
timestamp  count     0
name       count    11
geometry   count    23
Name: (401011001, medical), dtype: int64
osm_id     count    25
timestamp  count     0
name       count    

Name: (402041172, restaurant), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    0
geometry   count    1
Name: (402041172, tour), dtype: int64
osm_id     count    1
timestamp  count    0
name       count    0
geometry   count    1
Name: (402051049, culture), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    3
geometry   count    3
Name: (402051049, education), dtype: int64
osm_id     count    6
timestamp  count    0
name       count    2
geometry   count    6
Name: (402051049, government), dtype: int64
osm_id     count    4
timestamp  count    0
name       count    3
geometry   count    4
Name: (402051049, restaurant), dtype: int64
osm_id     count    3
timestamp  count    0
name       count    0
geometry   count    3
Name: (402051050, government), dtype: int64
osm_id     count    2
timestamp  count    0
name       count    0
geometry   count    2
Name: (402051050, medical), dtype: int64
osm_id     count    2
timestamp  count    0
n

osm_id     count    1
timestamp  count    0
name       count    1
geometry   count    1
Name: (404021101, education), dtype: int64
osm_id     count    5
timestamp  count    0
name       count    5
geometry   count    5
Name: (404021101, finance), dtype: int64
osm_id     count    27
timestamp  count     0
name       count     7
geometry   count    27
Name: (404021101, government), dtype: int64
osm_id     count    4
timestamp  count    0
name       count    0
geometry   count    4
Name: (404021101, information), dtype: int64
osm_id     count    20
timestamp  count     0
name       count    11
geometry   count    20
Name: (404021101, medical), dtype: int64
osm_id     count    55
timestamp  count     0
name       count    47
geometry   count    55
Name: (404021101, restaurant), dtype: int64
osm_id     count    4
timestamp  count    0
name       count    2
geometry   count    4
Name: (404021101, small_business), dtype: int64
osm_id     count    7
timestamp  count    0
name       count    3


In [34]:
poi_agg_df = pd.DataFrame()

In [35]:
poi_agg_df["SA2_MAIN16"] = count.index.values
poi_agg_df["poi_count"] = count[( 'geometry', 'count')].values

In [36]:
poi_agg_df.head()

,SA2_MAIN16,poi_count
0,401011001,643
1,401011002,113
2,401021003,42
3,401021004,124
4,401021005,50


In [37]:
split_count.shape

(732, 4)

In [38]:
import math

In [39]:
entropy = {}
for i, row in split_count.iterrows():
    sa_id, _type = i
    total_count = poi_agg_df.loc[poi_agg_df["SA2_MAIN16"]==sa_id]["poi_count"]
    val = row[( 'geometry', 'count')]/total_count
    
    if sa_id not in entropy:
        entropy[sa_id] = (-val * np.log(val))
    else:
        entropy[sa_id] += (-val * np.log(val))

In [40]:
print(len(entropy))
print(len(poi_agg_df))

124
124


In [41]:
entropy_list = []
for sa_id in poi_agg_df.SA2_MAIN16:
    entropy_list.append(float(entropy[sa_id]))

In [42]:
poi_agg_df["poi_count_entropy"] = entropy_list

In [43]:
poi_agg_df.head()

,SA2_MAIN16,poi_count,poi_count_entropy
0,401011001,643,1.550264
1,401011002,113,1.506143
2,401021003,42,1.453478
3,401021004,124,1.963282
4,401021005,50,1.488217


In [44]:
node_degree_entropy_df = node_degree_df.merge(poi_agg_df,how="left",on="SA2_MAIN16")

In [45]:
node_degree_entropy_df.head()

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,...,sum_stay_duration_destination_entropy,total_stays_destination_entropy,num_nodes,num_1degree,num_2degree,num_3degree,num_4degree,num_greater5degree,poi_count,poi_count_entropy
0,401011001,41001,Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,3.949646,4.237396,1642,291,199,865,276,11,643,1.550264
1,401011002,41002,North Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,3.152188,3.975034,517,149,54,239,69,6,113,1.506143
2,401021003,41003,Adelaide Hills,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.948900,2.042530,4034,1174,731,1982,141,6,42,1.453478
3,401021004,41004,Aldgate - Stirling,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,1.099284,2.075796,2455,710,502,1126,110,7,124,1.963282
4,401021005,41005,Hahndorf - Echunga,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,1.654530,2.549629,989,286,245,421,36,1,50,1.488217


In [46]:
# sw - editting poi_count and poit_count_entropy - replacing NaN by zeros
node_degree_entropy_df.loc[node_degree_entropy_df.poi_count.isna(), 'poi_count'] = 0.0 
node_degree_entropy_df.loc[node_degree_entropy_df.poi_count_entropy.isna(), 'poi_count_entropy'] = 0.0 


In [47]:
node_degree_entropy_df

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,...,sum_stay_duration_destination_entropy,total_stays_destination_entropy,num_nodes,num_1degree,num_2degree,num_3degree,num_4degree,num_greater5degree,poi_count,poi_count_entropy
0,401011001,41001,Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,3.949646,4.237396,1642,291,199,865,276,11,643,1.550264
1,401011002,41002,North Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,3.152188,3.975034,517,149,54,239,69,6,113,1.506143
2,401021003,41003,Adelaide Hills,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,0.948900,2.042530,4034,1174,731,1982,141,6,42,1.453478
3,401021004,41004,Aldgate - Stirling,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,1.099284,2.075796,2455,710,502,1126,110,7,124,1.963282
4,401021005,41005,Hahndorf - Echunga,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,1.654530,2.549629,989,286,245,421,36,1,50,1.488217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,404031105,41105,Fulham,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,1.975506,2.784143,157,28,19,90,20,0,9,1.303092
99,404031106,41106,Lockleys,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,1.416921,2.608267,530,102,39,308,81,0,18,1.581094
100,404031107,41107,Plympton,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,1.695853,2.732940,1036,170,105,616,143,2,38,1.495968
101,404031108,41108,Richmond (SA),40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,2.308701,3.390496,887,125,120,410,231,1,57,1.682331


In [48]:
node_degree_df.shape

(103, 48)

In [49]:
node_degree_entropy_df.shape

(103, 50)

In [80]:
node_degree_entropy_df[['poi_count','poi_count_entropy']]

,poi_count,poi_count_entropy
0,643,1.550264
1,113,1.506143
2,42,1.453478
3,124,1.963282
4,50,1.488217
...,...,...
98,9,1.303092
99,18,1.581094
100,38,1.495968
101,57,1.682331


In [107]:
# normalize
node_degree_entropy_df['poi_count_div_by_area']=node_degree_entropy_df['poi_count']/(node_degree_entropy_df.area/1000000)
node_degree_entropy_df['poi_count_entropy_div_by_area']=node_degree_entropy_df['poi_count_entropy']/(node_degree_entropy_df.area/1000000)


In [108]:
node_degree_entropy_df

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,...,num_4degree,num_greater5degree,poi_count,poi_count_entropy,poi_count_area,poi_count_entropy_area,poi_count_by_area,poi_count_by_entropy_area,poi_count_div_by_area,poi_count_entropy_div_by_area
0,401011001,41001,Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,276,11,643,1.550264,61.693702,0.148743,61.693702,0.148743,61.693702,0.148743
1,401011002,41002,North Adelaide,40101,Adelaide City,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,69,6,113,1.506143,22.326857,0.297588,22.326857,0.297588,22.326857,0.297588
2,401021003,41003,Adelaide Hills,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,141,6,42,1.453478,0.115969,0.004013,0.115969,0.004013,0.115969,0.004013
3,401021004,41004,Aldgate - Stirling,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,110,7,124,1.963282,1.063360,0.016836,1.063360,0.016836,1.063360,0.016836
4,401021005,41005,Hahndorf - Echunga,40102,Adelaide Hills,401,Adelaide - Central and Hills,4GADE,Greater Adelaide,4,...,36,1,50,1.488217,0.456135,0.013577,0.456135,0.013577,0.456135,0.013577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,404031105,41105,Fulham,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,20,0,9,1.303092,6.851511,0.992017,6.851511,0.992017,6.851511,0.992017
99,404031106,41106,Lockleys,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,81,0,18,1.581094,3.174781,0.278868,3.174781,0.278868,3.174781,0.278868
100,404031107,41107,Plympton,40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,143,2,38,1.495968,3.572091,0.140625,3.572091,0.140625,3.572091,0.140625
101,404031108,41108,Richmond (SA),40403,West Torrens,404,Adelaide - West,4GADE,Greater Adelaide,4,...,231,1,57,1.682331,6.422553,0.189559,6.422553,0.189559,6.422553,0.189559


In [109]:
import pickle
node_degree_entropy_df.to_pickle('../../data/processed_small_data/node_degree_entropy_df.pickle')

# with open('../../data/processed_small_data/node_degree_entropy_df.pickle', 'wb') as f:
#     f = pickle.dump(node_degree_entropy_df, f)
# with open('../../data/processed_small_data/node_degree_entropy_df.pickle', 'rb') as f:
#     file = pickle.load(f)


# Edge Degree DF

In [52]:
edge_degree_df

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,...,num_2degree_x,num_3degree_x,num_4degree_x,num_greater5degree_x,num_nodes_y,num_1degree_y,num_2degree_y,num_3degree_y,num_4degree_y,num_greater5degree_y
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,...,199.0,865.0,276.0,11.0,1642.0,291.0,199.0,865.0,276.0,11.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,...,199.0,865.0,276.0,11.0,517.0,149.0,54.0,239.0,69.0,6.0
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003,838.0,82.0,...,199.0,865.0,276.0,11.0,4034.0,1174.0,731.0,1982.0,141.0,6.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,...,199.0,865.0,276.0,11.0,2455.0,710.0,502.0,1126.0,110.0,7.0
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005,517.0,13.0,...,199.0,865.0,276.0,11.0,989.0,286.0,245.0,421.0,36.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,2018-02-01T00:00:00.000+11:00,404031105,4,404031109,2304,5876916,12172,404031109404031105,59.0,1.0,...,20.0,151.0,19.0,1.0,157.0,28.0,19.0,90.0,20.0,0.0
10236,2018-02-01T00:00:00.000+11:00,404031106,4,404031109,2967,5413932,16935,404031109404031106,105.0,1.0,...,20.0,151.0,19.0,1.0,530.0,102.0,39.0,308.0,81.0,0.0
10237,2018-02-01T00:00:00.000+11:00,404031107,4,404031109,1811,937583,8366,404031109404031107,262.0,1.0,...,20.0,151.0,19.0,1.0,1036.0,170.0,105.0,616.0,143.0,2.0
10238,2018-02-01T00:00:00.000+11:00,404031108,4,404031109,2464,1334692,11329,404031109404031108,182.0,1.0,...,20.0,151.0,19.0,1.0,887.0,125.0,120.0,410.0,231.0,1.0


In [53]:
entropy_x = []
notin = 0
for sa_id in edge_degree_df["sa2_code16_x"].values:
    if sa_id in entropy:
        entropy_x.append(float(entropy[sa_id]))
    else:
        notin += 1
print(notin)
notin=0
entropy_y = []
for sa_id in edge_degree_df["sa2_code16_y"]:
    if sa_id in entropy:
        entropy_y.append(float(entropy[sa_id]))
    else:
        notin += 1
print(notin)

0
0


In [54]:
notin

0

In [55]:
edge_degree_df.shape

(10240, 39)

In [56]:
poi_agg_df.head()

,SA2_MAIN16,poi_count,poi_count_entropy
0,401011001,643,1.550264
1,401011002,113,1.506143
2,401021003,42,1.453478
3,401021004,124,1.963282
4,401021005,50,1.488217


In [57]:
sa_ids_poi = set(poi_agg_df["SA2_MAIN16"].values)

In [58]:
edge_degree_df = edge_degree_df[edge_degree_df["sa2_code16_y"].isin(sa_ids_poi)]

In [59]:
print(edge_degree_df.shape)

(10240, 39)


In [60]:
edge_degree_df = edge_degree_df[edge_degree_df["sa2_code16_x"].isin(sa_ids_poi)]

In [61]:
print(edge_degree_df.shape)

(10240, 39)


In [62]:
count_dic = {key:val for key,val in zip(poi_agg_df["SA2_MAIN16"].values, poi_agg_df["poi_count"].values)}

In [63]:
entropy_x = []
poi_count_x = []
notin = 0
for sa_id in edge_degree_df["sa2_code16_x"].values:
    if sa_id in entropy:
        entropy_x.append(float(entropy[sa_id]))
        poi_count_x.append(float(count_dic[sa_id]))
    else:
        notin += 1
print(notin)
notin=0
entropy_y = []
poi_count_y = []
for sa_id in edge_degree_df["sa2_code16_y"]:
    if sa_id in entropy:
        entropy_y.append(float(entropy[sa_id]))
        poi_count_y.append(float(count_dic[sa_id]))
    else:
        notin += 1
print(notin)

0
0


In [64]:
print(len(entropy_x))
print(len(entropy_y))

10240
10240


In [65]:
edge_degree_df["poi_entropy_x"] = entropy_x
edge_degree_df["poi_entropy_y"] = entropy_y

In [66]:
edge_degree_df["poi_count_x"] = poi_count_x
edge_degree_df["poi_count_y"] = poi_count_y

In [67]:
edge_degree_df.shape

(10240, 43)

In [68]:
edge_degree_df.head()

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,...,num_nodes_y,num_1degree_y,num_2degree_y,num_3degree_y,num_4degree_y,num_greater5degree_y,poi_entropy_x,poi_entropy_y,poi_count_x,poi_count_y
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,...,1642.0,291.0,199.0,865.0,276.0,11.0,1.550264,1.550264,643.0,643.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,...,517.0,149.0,54.0,239.0,69.0,6.0,1.550264,1.506143,643.0,113.0
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003,838.0,82.0,...,4034.0,1174.0,731.0,1982.0,141.0,6.0,1.550264,1.453478,643.0,42.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,...,2455.0,710.0,502.0,1126.0,110.0,7.0,1.550264,1.963282,643.0,124.0
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005,517.0,13.0,...,989.0,286.0,245.0,421.0,36.0,1.0,1.550264,1.488217,643.0,50.0


In [69]:
# # sw. Add the current edge_degree_df and the initial edge_degree_df.pickle 
# init_edge_degree_df = pd.read_pickle('../../data/processed_small_data/edge_degree_df.pickle')
# init_edge_degree_df

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,...,num_2degree_x,num_3degree_x,num_4degree_x,num_greater5degree_x,num_nodes_y,num_1degree_y,num_2degree_y,num_3degree_y,num_4degree_y,num_greater5degree_y
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,...,199.0,865.0,276.0,11.0,1642.0,291.0,199.0,865.0,276.0,11.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,...,199.0,865.0,276.0,11.0,517.0,149.0,54.0,239.0,69.0,6.0
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003,838.0,82.0,...,199.0,865.0,276.0,11.0,4034.0,1174.0,731.0,1982.0,141.0,6.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,...,199.0,865.0,276.0,11.0,2455.0,710.0,502.0,1126.0,110.0,7.0
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005,517.0,13.0,...,199.0,865.0,276.0,11.0,989.0,286.0,245.0,421.0,36.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,2018-02-01T00:00:00.000+11:00,404031105,4,404031109,2304,5876916,12172,404031109404031105,59.0,1.0,...,20.0,151.0,19.0,1.0,157.0,28.0,19.0,90.0,20.0,0.0
10236,2018-02-01T00:00:00.000+11:00,404031106,4,404031109,2967,5413932,16935,404031109404031106,105.0,1.0,...,20.0,151.0,19.0,1.0,530.0,102.0,39.0,308.0,81.0,0.0
10237,2018-02-01T00:00:00.000+11:00,404031107,4,404031109,1811,937583,8366,404031109404031107,262.0,1.0,...,20.0,151.0,19.0,1.0,1036.0,170.0,105.0,616.0,143.0,2.0
10238,2018-02-01T00:00:00.000+11:00,404031108,4,404031109,2464,1334692,11329,404031109404031108,182.0,1.0,...,20.0,151.0,19.0,1.0,887.0,125.0,120.0,410.0,231.0,1.0


In [70]:
# # keep all the observations
# init_edge_degree_df = init_edge_degree_df.merge(edge_degree_df[['poi_entropy_x', 'poi_entropy_y', 'poi_count_x', 'poi_count_y']], left_index = True, right_index = True, how = 'outer')


In [71]:
# # replace NaN by zeros
# init_edge_degree_df.loc[init_edge_degree_df.poi_entropy_x.isna(), ['poi_entropy_x', 'poi_entropy_y', 'poi_count_x', 'poi_count_y']] = 0.0


In [72]:
# init_edge_degree_df

,timestamp,destination,state,origin,unique_agents,sum_stay_duration,total_stays,OD,class_ART,class_BUS,...,num_nodes_y,num_1degree_y,num_2degree_y,num_3degree_y,num_4degree_y,num_greater5degree_y,poi_entropy_x,poi_entropy_y,poi_count_x,poi_count_y
0,2018-02-01T00:00:00.000+11:00,401011001,4,401011001,19863,268779414,681217,401011001401011001,208.0,13.0,...,1642.0,291.0,199.0,865.0,276.0,11.0,1.550264,1.550264,643.0,643.0
1,2018-02-01T00:00:00.000+11:00,401011002,4,401011001,6911,4857456,29322,401011001401011002,267.0,16.0,...,517.0,149.0,54.0,239.0,69.0,6.0,1.550264,1.506143,643.0,113.0
2,2018-02-01T00:00:00.000+11:00,401021003,4,401011001,404,253764,1334,401011001401021003,838.0,82.0,...,4034.0,1174.0,731.0,1982.0,141.0,6.0,1.550264,1.453478,643.0,42.0
3,2018-02-01T00:00:00.000+11:00,401021004,4,401011001,1184,903377,5487,401011001401021004,430.0,13.0,...,2455.0,710.0,502.0,1126.0,110.0,7.0,1.550264,1.963282,643.0,124.0
4,2018-02-01T00:00:00.000+11:00,401021005,4,401011001,700,189721,1278,401011001401021005,517.0,13.0,...,989.0,286.0,245.0,421.0,36.0,1.0,1.550264,1.488217,643.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,2018-02-01T00:00:00.000+11:00,404031105,4,404031109,2304,5876916,12172,404031109404031105,59.0,1.0,...,157.0,28.0,19.0,90.0,20.0,0.0,1.270018,1.303092,14.0,9.0
10236,2018-02-01T00:00:00.000+11:00,404031106,4,404031109,2967,5413932,16935,404031109404031106,105.0,1.0,...,530.0,102.0,39.0,308.0,81.0,0.0,1.270018,1.581094,14.0,18.0
10237,2018-02-01T00:00:00.000+11:00,404031107,4,404031109,1811,937583,8366,404031109404031107,262.0,1.0,...,1036.0,170.0,105.0,616.0,143.0,2.0,1.270018,1.495968,14.0,38.0
10238,2018-02-01T00:00:00.000+11:00,404031108,4,404031109,2464,1334692,11329,404031109404031108,182.0,1.0,...,887.0,125.0,120.0,410.0,231.0,1.0,1.270018,1.682331,14.0,57.0


In [115]:
# normalize poi_entropy_x poi_entropy_y poi_count_x poi_count_y.
# node_degree_entropy_df.SA2_MAIN16
area_mapping_dic = dict(zip(node_degree_entropy_df.SA2_MAIN16.values, node_degree_entropy_df.area.values/1000000))
area_mapping_dic

# 
edge_degree_df['origin_area']= edge_degree_df['origin'].map(area_mapping_dic)
edge_degree_df['destination_area']= edge_degree_df['destination'].map(area_mapping_dic)

# normalize 
edge_degree_df['poi_entropy_x_div_by_area']=edge_degree_df['poi_entropy_x']/edge_degree_df['origin_area']
edge_degree_df['poi_entropy_y_div_by_area']=edge_degree_df['poi_entropy_y']/edge_degree_df['destination_area']
edge_degree_df['poi_count_x_div_by_area']=edge_degree_df['poi_count_x']/edge_degree_df['origin_area']
edge_degree_df['poi_count_y_div_by_area']=edge_degree_df['poi_count_y']/edge_degree_df['destination_area']


In [126]:
# address na
edge_degree_df.dropna(how='any',inplace = True) 

In [130]:
np.sum(edge_degree_df.isin([np.inf, -np.inf]))

timestamp                                0
destination                              0
state                                    0
origin                                   0
unique_agents                            0
sum_stay_duration                        0
total_stays                              0
OD                                       0
class_ART                                0
class_BUS                                0
class_COLL                               0
class_FREE                               0
class_HWY                                0
class_LOCL                               0
class_SUBA                               0
class_TRK2                               0
class_TRK4                               0
class_UND                                0
num_jobs_000_persons_origin              0
sa2_code16_x                             0
median_income_per_job_aud_origin         0
num_jobs_000_persons_destination         0
sa2_code16_y                             0
median_inco

In [128]:
edge_degree_df.to_pickle('../../data/processed_small_data/edge_degree_entropy_df.pickle')
